In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import yfinance as yf
import scipy
from scipy.stats import norm
import math
from math import *

btc = yf.download('BTC-USD', start='2020-01-01', interval='1d')
eth = yf.download('ETH-USD', start='2020-01-01', interval='1d')
btc_basis_result = pd.read_excel('C:\\Users\\offic\\Downloads\\crypto.xlsx', sheet_name='btc')
eth_basis_result = pd.read_excel('C:\\Users\\offic\\Downloads\\crypto.xlsx', sheet_name='eth')


def cdf(x):
    return (1.0 + erf(x / sqrt(2.0))) / 2.0


def phi(s, t, gamma, h, i, r, a, v):
    lambda1 = (-r + gamma * a + 0.5 * gamma * (gamma - 1) * v**2) * t
    dd = -(log(s / h) + (a + (gamma - 0.5) * v**2) * t) / (v * sqrt(t))
    k = 2 * a / (v**2) + (2 * gamma - 1)

    try:
        return exp(lambda1) * s**gamma * (cdf(dd) - (i / s)**k * cdf(dd - 2 * log(i / s) / (v * sqrt(t))))
    except OverflowError as err:
        return exp(lambda1) * s**gamma * cdf(dd)


def bjerksund_stensland_call(underlying_price, exercise_price, time_in_years, risk_free_rate, volatility):
    div = 1e-08
    z = 1
    rr = risk_free_rate
    dd2 = div
    
    dt = volatility * sqrt(time_in_years)
    drift = risk_free_rate - div
    v2 = volatility**2
    
    b1 = sqrt((z * drift / v2 - 0.5)**2 + 2 * rr / v2)
    beta = (1 / 2 - z * drift / v2) + b1
    binfinity = beta / (beta - 1) * exercise_price
    bb = max(exercise_price, rr / dd2 * exercise_price)
    ht = -(z * drift * time_in_years + 2 * dt) * bb / (binfinity - bb)
    i = bb + (binfinity - bb) * (1 - exp(ht))

    if underlying_price < i and beta < 100:
        alpha = (i - exercise_price) * i**(-beta)
        return alpha * underlying_price**beta - alpha * phi(underlying_price, time_in_years, beta, i, i, rr, z * drift, volatility) + phi(underlying_price, time_in_years, 1, i, i, rr, z * drift, volatility) - phi(underlying_price, time_in_years, 1, exercise_price, i, rr, z * drift, volatility) - exercise_price * phi(underlying_price, time_in_years, 0, i, i, rr, z * drift, volatility) + exercise_price * phi(underlying_price, time_in_years, 0, exercise_price, i, rr, z * drift, volatility)
    
    return underlying_price - exercise_price


def bjerksund_stensland_put(underlying_price, exercise_price, time_in_years, risk_free_rate, volatility):
    div = 1E-08
    z = -1
    rr = div
    dd = rr
    dd2 = 2 * dd - rr
    asset_new = underlying_price
    underlying_price = exercise_price
    exercise_price = asset_new

    dt = volatility * sqrt(time_in_years)
    drift = risk_free_rate - div
    v2 = volatility**2

    b1 = sqrt((z * drift / v2 - 0.5)**2 + 2 * rr / v2)
    beta = (1 / 2 - z * drift / v2) + b1
    binfinity = beta / (beta - 1) * exercise_price
    bb = max(exercise_price, rr / dd2 * exercise_price)
    ht = -(z * drift * time_in_years + 2 * dt) * bb / (binfinity - bb)
    i = bb + (binfinity - bb) * (1 - exp(ht))
        
    if underlying_price < i and beta < 100: # To avoid overflow
        alpha = (i - exercise_price) * i**(-beta)
        return alpha * underlying_price**beta - alpha * phi(underlying_price, time_in_years, beta, i, i, rr, z * drift, volatility) + phi(underlying_price, time_in_years, 1, i, i, rr, z * drift, volatility) - phi(underlying_price, time_in_years, 1, exercise_price, i, rr, z * drift, volatility) - exercise_price * phi(underlying_price, time_in_years, 0, i, i, rr, z * drift, volatility) + exercise_price * phi(underlying_price, time_in_years, 0, exercise_price, i, rr, z * drift, volatility)
    
    return underlying_price - exercise_price


def hv_at_the_moment(df, duration, iterator):
    df_yahoo = df[:duration * 2 + iterator]
    high = df_yahoo['High']
    low = df_yahoo['Low']
    close = df_yahoo['Close']
    in_day = 1
    window_1 = int(duration * in_day)
    close_to_close = (
        close.pct_change(periods=1).rolling(window=window_1).std() * 100 * np.sqrt(365 * in_day)
    )
    high_low = (
        (
            0.627 * np.sqrt(365.25) * np.log(high / low)
        ).ewm(span=window_1, adjust=False).mean() / np.sqrt(365) * 100 * np.sqrt(365 * in_day)
    )
    my_vol_1 = round((high_low * 0.8 + close_to_close * 0.2) * 1, 2)

    window_2 = int(duration * in_day + duration * in_day * 0.5)
    close_to_close = (
        close.pct_change(periods=1).rolling(window=window_2).std() * 100 * np.sqrt(365 * in_day)
    )
    high_low = (
        (
            0.627 * np.sqrt(365.25) * np.log(high / low)
        ).ewm(span=window_2, adjust=False).mean() / np.sqrt(365) * 100 * np.sqrt(365 * in_day)
    )
    my_vol_2 = round((high_low * 0.8 + close_to_close * 0.2) * 1, 2)

    window_3 = int(duration * in_day - duration * in_day * 0.5)
    close_to_close = (
        close.pct_change(periods=1).rolling(window=window_3).std() * 100 * np.sqrt(365 * in_day)
    )
    high_low = (
        (
            0.627 * np.sqrt(365.25) * np.log(high / low)
        ).ewm(span=window_3, adjust=False).mean() / np.sqrt(365) * 100 * np.sqrt(365 * in_day)
    )
    my_vol_3 = round((high_low * 0.8 + close_to_close * 0.2) * 1, 2)
    my_vol = (my_vol_1 * 0.5 + my_vol_2 * 0.25 + my_vol_3 * 0.25)[-1] / 100
    return my_vol


def df_result(df_yahoo, df_basis, percent, opt_type):
    date_list = []
    result_list = []
    iter_1 = 0
    iter_2 = 0
    for i in range(0, len(df_basis), 1):
        price = df_basis['price_in'][iter_1]
        price_in_dur = df_basis['price_out'][iter_1]
        date = df_basis['close_date'][iter_1]
        strike = price + price * percent
        duration = df_basis['days'][iter_1]
        basis_result = df_basis['result'][iter_1]
        hv = hv_at_the_moment(df_yahoo, duration, iter_1)
        opt_price = bjerksund_stensland_call(
            price,
            strike,
            duration / 365,
            0.04,
            hv
        )
        if opt_type == 'C':
            be = strike + opt_price
            if price_in_dur >= be:
                result = (be - price_in_dur + basis_result) / 100 * 11
            else:
                result = (opt_price + basis_result) / 100 * 11
        elif opt_type == 'P':
            be = strike - opt_price
            if price_in_dur >= be:
                result = (opt_price + basis_result) / 100 * 11
            else:
                result = (price_in_dur - be + basis_result) / 100 * 11
        date_list.append(date)
        result_list.append(result)
        iter_1 += 1
    df_result = pd.DataFrame({})
    df_result['Date'] = date_list
    df_result['Result'] = result_list
    df_result['Result cumsum'] = df_result['Result'].cumsum()
    return df_result


def get_chart(df, title):
    fig = px.line(
        df,
        x='Date',
        y='Result cumsum',
        width=950,
        height=650,
        title=title
    )
    fig.show()


df_btc_result_call = df_result(df_yahoo=btc, df_basis=btc_basis_result, percent=0.04, opt_type='C')
df_eth_result_call = df_result(df_yahoo=eth, df_basis=eth_basis_result, percent=0.04, opt_type='C')

df_btc_result_put = df_result(df_yahoo=btc, df_basis=btc_basis_result, percent=0.04, opt_type='P')
df_eth_result_put = df_result(df_yahoo=eth, df_basis=eth_basis_result, percent=0.04, opt_type='P')

get_chart(df_btc_result_call, 'BTC Covered CALL')
get_chart(df_btc_result_put, 'BTC Cash-Secured PUT')

get_chart(df_eth_result_call, 'ETH Covered CALL')
get_chart(df_eth_result_put, 'ETH Cash-Secured PUT')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [40]:
price = 100
strike = 105
iv = 0.2
rate = 0.055
t = 30 / 365
d_1 = (np.log(price / strike) + (0.5 * (iv**2) + rate) * t) / (iv * t**0.5)
delta = norm.cdf(d_1)
delta

0.22861687240813666

In [42]:
ph = norm.cdf(iv)
price * math.exp(-(ph**-1 * delta) * iv * t**0.5 + (0.5 * iv**2 + rate) * t)

98.36694779586959